# Kernel Conservation Report

This notebook sweeps kernel parameters and evaluates:

- Per-step **energy drift** (conservation check)
- **Particle** and **field** yields
- Heatmaps over phase and cancellation efficiency

**Prereqs**: Your patched kernel (`PTKKernel`, `Sound`, `EmissionConfig`) and `ptk.v1.json` should be available. If your kernel lives at `pt-sim/pt_sim/ptk_kernel.py`, this notebook will import it. Otherwise it falls back to a local `ptk_kernel_patched.py` if present.

In [ ]:
# Imports & setup
import os, sys, json, math, numpy as np
import matplotlib.pyplot as plt

# Try to import kernel from package; fallback to a local file if needed
KernelSource = None
try:
    from pt_sim.ptk_kernel import PTKKernel, Sound, EmissionConfig
    KernelSource = 'pt_sim.ptk_kernel'
except Exception as e:
    print('[Info] Could not import pt_sim.ptk_kernel:', e)
    if os.path.exists(os.path.join(os.getcwd(), 'ptk_kernel_patched.py')):
        from ptk_kernel_patched import PTKKernel, Sound, EmissionConfig
        KernelSource = 'ptk_kernel_patched.py'
    else:
        raise RuntimeError('Kernel not found. Ensure pt-sim/pt_sim/ptk_kernel.py is importable or place ptk_kernel_patched.py next to this notebook.')

print('Using kernel from:', KernelSource)

# Load PTK kernel JSON (adjust path if needed)
PTK_JSON_PATH = os.environ.get('PTK_JSON_PATH', 'ptk.v1.json')
with open(PTK_JSON_PATH, 'r', encoding='utf-8') as f:
    ptk = json.load(f)
K = PTKKernel(ptk)
print('Loaded PTK with', len(ptk['nodes']), 'nodes and', len(ptk['edges']), 'edges')

## Single-run helper
Runs one emission with a given phase and cancellation efficiency, returns the kernel result + per-step drift array.

In [ ]:
def run_once(phase, cancel_eff=0.7, max_steps=20, seed=123):
    pos = [Sound('n1', +1, 5.0, 440.0, 0.0, 0.9, 'P1')]
    neg = [Sound('n4', -1, 5.0, 440.0, phase, 0.9, 'N1')]
    cfg = EmissionConfig(max_steps=max_steps, cancel_efficiency=cancel_eff, conservation_tol=1e-4, rng_seed=seed)
    res = K.simulate_emission(pos, neg, cfg)
    drift = np.array([s.get('drift_rel', 0.0) for s in res['ledger']['steps']], dtype=float)
    return res, drift

## 1D sweep: phase ∈ [0, 2π]
We vary relative phase between + and − lobes and track conservation drift and yields.

In [ ]:
phases = np.linspace(0, 2*np.pi, 25)
drift_means, y_particles, y_fields = [], [], []
for ph in phases:
    res, drift = run_once(ph)
    drift_means.append(float(drift.mean()) if drift.size else 0.0)
    y_particles.append(len(res['particles']))
    y_fields.append(len(res['fields']))

plt.figure(figsize=(12,4))
plt.subplot(1,3,1); plt.plot(phases, drift_means); plt.title('Mean drift vs phase'); plt.xlabel('phase [rad]'); plt.ylabel('mean drift')
plt.subplot(1,3,2); plt.plot(phases, y_particles); plt.title('#particles vs phase'); plt.xlabel('phase [rad]')
plt.subplot(1,3,3); plt.plot(phases, y_fields); plt.title('#fields vs phase'); plt.xlabel('phase [rad]')
plt.tight_layout(); plt.show()

## 2D sweep: (phase, cancel_efficiency)
Heatmaps for particle count, field count, and mean drift over a grid.

In [ ]:
cancs = np.linspace(0.4, 0.9, 9)
grid_particles = np.zeros((len(cancs), len(phases)), dtype=float)
grid_fields    = np.zeros_like(grid_particles)
grid_drift     = np.zeros_like(grid_particles)

for i, c_eff in enumerate(cancs):
    for j, ph in enumerate(phases):
        res, drift = run_once(ph, cancel_eff=c_eff)
        grid_particles[i, j] = len(res['particles'])
        grid_fields[i, j]    = len(res['fields'])
        grid_drift[i, j]     = float(drift.mean()) if drift.size else 0.0

def imshow_grid(G, title):
    plt.figure(figsize=(8,4))
    plt.imshow(G, aspect='auto', origin='lower',
               extent=[phases.min(), phases.max(), cancs.min(), cancs.max()])
    plt.colorbar(); plt.title(title); plt.xlabel('phase [rad]'); plt.ylabel('cancel_efficiency')
    plt.show()

imshow_grid(grid_particles, '#particles')
imshow_grid(grid_fields, '#fields')
imshow_grid(grid_drift, 'mean drift')

## Energy accounting snapshot
Peek into the ledger for one configuration to verify fields like `packets_energy_before/after`, `drift_rel`, etc.

In [ ]:
res_one, drift_one = run_once(phase=math.pi)
print('Steps:', res_one['steps'])
print('First 3 step records:')
for s in res_one['ledger']['steps'][:3]:
    print(s)
print('Final ledger:', res_one['ledger']['final'])